In [52]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
file = uproot.open("/home/zhonghua/Filt_Event/model_validation/exposure.root")
h1 = file["h1"]
h2 = file["h2"]
# 获取直方图的所有数据
values = h1.values()        # bin内容（纵坐标）
h2_values = h2.values()        # bin内容（纵坐标）
edges = h1.axis().edges()   # bin边界（横坐标）
h2_edges = h2.axis().edges()   # bin边界（横坐标）
centers = h1.axis().centers()  # bin中心
h2_centers = h2.axis().centers()  # bin中心

values = np.array(h1.values())
edges = np.array(h1.axis().edges())
centers = np.array(h1.axis().centers())
S=np.pi * 1000**2
parker=1e-15 * 1e4 
nums=[]
for i, theta in enumerate(centers):
    cosine_theta = np.cos(np.deg2rad(theta))
    area = S * cosine_theta
    exposure = values[i] * area
    num_i = exposure * parker
    nums.append(num_i)

weights=[num_i/251000/h2_values[i]/86400 for i, num_i in enumerate(nums)]

# 加载磁单极子的数据 /home/zhonghua/data/Dataset_Filted/1e10_V03/Monopole_combined_filted_1e10_V03_dataset.npz
monopole_data=np.load("/home/zhonghua/data/Dataset_Filted/filted_Monopole_1e10_merged_1000_70_new_dataset.npz",allow_pickle=True)
monopole_params=monopole_data["params"]
params_name=monopole_data["param_names"]
monopole_df=pd.DataFrame(monopole_params,columns=params_name)
theta_degrees = monopole_df["rec_theta"] * 180 / np.pi
monopole_weights = []
for theta in theta_degrees:
    if theta > max(centers) or theta < 0:
        weight = weights[-1] if theta > max(centers) else 0
    else:
        bin_id = np.digitize(theta, edges) - 1
        weight = weights[bin_id]
    monopole_weights.append(weight)

monopole_df["weight"] = monopole_weights
print(len(monopole_df["weight"].values),)
print(f"one day exposure: {np.sum(monopole_df['weight'].values)*86400} ")
print(f"one year exposure: {np.sum(monopole_df['weight'].values)*86400*365}")

# 1e10
log10TeV_low=1.7969200611114502
log10TeV_high=2.1131057739257812
monopole_recx = pd.to_numeric(monopole_df["rec_x"].values, errors='coerce')
monopole_recy = pd.to_numeric(monopole_df["rec_y"].values, errors='coerce')
monopole_df["rec_r"]=np.sqrt(monopole_recx**2+monopole_recy**2)
monopole_x = pd.to_numeric(monopole_df["core_x"].values, errors='coerce')
monopole_y = pd.to_numeric(monopole_df["core_y"].values, errors='coerce')
monopole_df["r"]=np.sqrt(monopole_x**2+monopole_y**2)
r_cut_low=300
r_cut_high=500
theta_cut_low=0/180*np.pi
theta_cut_high=50/180*np.pi

monopole_df=monopole_df[(monopole_df["rec_r"]>r_cut_low) & (monopole_df["rec_r"]<r_cut_high) 
                                & (monopole_df["rec_theta"]>theta_cut_low) & (monopole_df["rec_theta"]<theta_cut_high)
                                &(monopole_df["NpE3"]>80) & (monopole_df['NuW2']>0) 
                                ]
# monopole_df=monopole_df[(monopole_df["NpE3"]>80) & (monopole_df['NuW2']>0)]
print(len(monopole_df["weight"].values),)
print(f"one day exposure: {np.sum(monopole_df['weight'].values)*86400} ")
print(f"one year exposure: {np.sum(monopole_df['weight'].values)*86400*365}")

print("经过角度以及半径, 以及NpE NuW2, 以及重建能量的筛选")
print(monopole_df.columns)


104840
one day exposure: 2.598665026962714 
one year exposure: 948.5127348413905
26714
one day exposure: 0.8054541636769832 
one year exposure: 293.99076974209885
经过角度以及半径, 以及NpE NuW2, 以及重建能量的筛选
Index(['label', 'weight', 'R_ue', 'R_mean', 'Eage', 'recE', 'rec_x', 'rec_y',
       'rec_theta', 'rec_phi', 'trueE', 'NuW2', 'NpE3', 'NuM1', 'core_x',
       'core_y', 'theta', 'rec_r', 'r'],
      dtype='object')


In [49]:
#  rec_theta in [theta_cut_low, theta_cut_high] and rec_r in [r_cut_low, r_cut_high]
#  but theta or r not in [theta_cut_low, theta_cut_high] or [r_cut_low, r_cut_high]
#  df :monopole_df
def pollut_indices(df, theta_cut_low, theta_cut_high, r_cut_low, r_cut_high):
    """
    找到污染事件的索引
    """
    # rec_theta 和 rec_r 在范围内
    rec_in_range = ((df['rec_theta'] >= theta_cut_low) & (df['rec_theta'] <= theta_cut_high) &
                   (df['rec_r'] >= r_cut_low) & (df['rec_r'] <= r_cut_high))
    
    # theta 或 r 不在范围内
    original_out_of_range = ((df['theta'] < theta_cut_low) | (df['theta'] > theta_cut_high) |
                           (df['r'] < r_cut_low) | (df['r'] > r_cut_high))
    
    # 综合条件
    condition = rec_in_range & original_out_of_range
    
    return df[condition].index.values        

indices = pollut_indices(monopole_df, theta_cut_low, theta_cut_high, r_cut_low, r_cut_high) 
print(len(indices)/len(monopole_df))


0.006288837313768061


In [50]:

age_low=0.8; age_high=1.2; R_mean_low=20; R_mean_high=60;R_ue_cut=-2.5
mn_df_age = monopole_df[(monopole_df['Eage']>=age_low)&(monopole_df['Eage']<=age_high)]
mn_df_Rmean = mn_df_age[(mn_df_age['R_mean']>=R_mean_low)&(mn_df_age['R_mean']<=R_mean_high)]
mn_df_Rue = mn_df_Rmean[(mn_df_Rmean['R_ue']<R_ue_cut)]

print(f"When age:{age_low}~{age_high}:")
print(f"Monopole: {np.sum(mn_df_age['weight'])}, left ratio: {np.sum(mn_df_age['weight'])/np.sum(monopole_df['weight']):.2e}")
print()
print(f"When R_mean:{R_mean_low}~{R_mean_high}:")
print(f"Monopole: {np.sum(mn_df_Rmean['weight'])}, left ratio: {np.sum(mn_df_Rmean['weight'])/np.sum(mn_df_age['weight']):.2e}")
print()
print(f"When R_ue<{R_ue_cut}:")
print(f"Monopole: {np.sum(mn_df_Rue['weight'])}, left ratio: {np.sum(mn_df_Rue['weight'])/np.sum(mn_df_Rmean['weight']):.2e}")
print()

indices_pollut = pollut_indices(mn_df_Rue, theta_cut_low, theta_cut_high, r_cut_low, r_cut_high) 
print(len(indices_pollut)/len(mn_df_Rue))


When age:0.8~1.2:
Monopole: 9.27294053178417e-06, left ratio: 9.95e-01

When R_mean:20~60:
Monopole: 9.27294053178417e-06, left ratio: 1.00e+00

When R_ue<-2.5:
Monopole: 7.678788318269349e-06, left ratio: 8.28e-01

0.0062613430127041745
